In [1]:
from dotenv import load_dotenv
load_dotenv()

import openai


# LangChain: Models, Prompts and Output Parsers


## Outline

 * Direct API calls to OpenAI
 * API calls through LangChain:
   * Prompts
   * Models
   * Output parsers
   

In [8]:
llm_model = "gpt-3.5-turbo"

from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt_template = ChatPromptTemplate.from_template(template_string)

chat = ChatOpenAI(temperature=0.0, model=llm_model)


customer_style = """American English \
in a calm and respectful tone
"""

customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""



In [6]:
response = chat.invoke("What is 1+1?")

print(response.content)

1+1 equals 2.


In [12]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

print(type(customer_messages))
print(type(customer_messages[0]))
print(customer_messages[0].content)

<class 'list'>
<class 'langchain_core.messages.human.HumanMessage'>
Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone
. text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```



In [13]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [14]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

In [17]:
customer_review = """\
Este soplador de hojas es bastante increíble. Tiene cuatro configuraciones:\
soplador de velas, brisa suave, ciudad ventosa y tornado. \
Llegó en dos días, justo a tiempo para el regalo del \
aniversario de mi esposa. \
Creo que a mi esposa le gustó tanto que se quedó sin palabras. \
Hasta ahora he sido el único que lo usa, y lo he estado \
usando cada dos mañanas para limpiar las hojas de nuestro césped. \
Es un poco más caro que otros sopladores de hojas \
que hay por ahí, pero creo que vale la pena por las características adicionales.
"""

review_template = """\
Para el siguiente texto, extrae la siguiente información:

gift: ¿El artículo fue comprado como regalo para otra persona? \
Responde True si es así, False si no o desconocido.

delivery_days: ¿Cuántos días tardó el producto \
en llegar? Si esta información no se encuentra, devuelve -1.

price_value: Extrae cualquier oración sobre el valor o precio,\
y devuélvelas como una lista de Python separada por comas.

Formatea la salida como JSON con las siguientes claves:
gift
delivery_days
price_value

text: {text}
"""

In [15]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [22]:
format_instructions = output_parser.get_format_instructions()

review_template_2 = """\
Para el siguiente texto, extrae la siguiente información:

gift: ¿El artículo fue comprado como regalo para otra persona? \
Responde True si es así, False si no o desconocido.

delivery_days: ¿Cuántos días tardó el producto\
en llegar? Si esta información no se encuentra, devuelve -1.

price_value: Extrae cualquier oración sobre el valor o precio,\
y devuélvelas como una lista de Python separada por comas.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [23]:
print(messages[0].content)

Para el siguiente texto, extrae la siguiente información:

gift: ¿El artículo fue comprado como regalo para otra persona? Responde True si es así, False si no o desconocido.

delivery_days: ¿Cuántos días tardó el productoen llegar? Si esta información no se encuentra, devuelve -1.

price_value: Extrae cualquier oración sobre el valor o precio,y devuélvelas como una lista de Python separada por comas.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing 

In [32]:
output_dict = output_parser.parse(text=messages[0].content)


OutputParserException: Got invalid return object. Expected key `gift` to be present, but got  and 
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

In [29]:
response = chat.invoke(messages)
response.content


'```json\n{\n\t"gift": True,\n\t"delivery_days": 2,\n\t"price_value": ["It\'s slightly more expensive than the other leaf blowers out there, but I think it\'s worth it for the extra features."]\n}\n```'